In [ ]:
from ast import alias
from concurrent.futures import process
from django.shortcuts import render
# Create your views here.
from django.shortcuts import render, HttpResponse
from django.contrib import messages
import Automatic_English_Essay_Scoring_Algorithm_Based_On_Ml
from .forms import UserRegistrationForm
from .models import UserRegistrationModel
from django.conf import settings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import datetime as dt
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import classification_report
# Create your views here.
def UserRegisterActions(request):
    if request.method == 'POST':
        form = UserRegistrationForm(request.POST)
        if form.is_valid():
            print('Data is Valid')
            form.save()
            messages.success(request, 'You have been successfully registered')
            form = UserRegistrationForm()
            return render(request, 'UserRegistrations.html', {'form': form})
        else:
            messages.success(request, 'Email or Mobile Already Existed')
            print("Invalid form")
    else:
        form = UserRegistrationForm()
    return render(request, 'UserRegistrations.html', {'form': form})
def UserLoginCheck(request):
    if request.method == "POST":
        loginid = request.POST.get('loginid')
        pswd = request.POST.get('pswd')
        print("Login ID = ", loginid, ' Password = ', pswd)
        try:
            check = UserRegistrationModel.objects.get(
                loginid=loginid, password=pswd)
            status = check.status
            print('Status is = ', status)
            if status == "activated":
                request.session['id'] = check.id
                request.session['loggeduser'] = check.name
                request.session['loginid'] = loginid
                request.session['email'] = check.email
                print("User id At", check.id, status)
                return render(request, 'users/UserHomePage.html', {})
            else:
                messages.success(request, 'Your Account Not at activated')
                return render(request, 'UserLogin.html')
        except Exception as e:
            print('Exception is ', str(e))
            pass
        messages.success(request, 'Invalid Login id and password')
    return render(request, 'UserLogin.html', {})
def UserHome(request):
    return render(request, 'users/UserHomePage.html', {})
def DatasetView(request):
    df = pd.read_csv("C:\\Users\\Admin\\Downloads\\training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
    df.dropna(axis=1,inplace=True)
    df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
    df.head()
    temp = pd.read_csv("C:\\Users\\Admin\\Downloads\\Processed_data.csv")
    temp.drop("Unnamed: 0",inplace=True,axis=1)
    return render(request, 'users/viewdataset.html', {'data': df})
def training(request):
    import numpy as np
    import pandas as pd
    import nltk
    import re
    from nltk.corpus import stopwords
    from nltk.tokenize import sent_tokenize,word_tokenize
    from gensim.models import Word2Vec
    from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
    from keras.models import Sequential, load_model, model_from_config
    import keras.backend as K
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import cohen_kappa_score
    df = pd.read_csv("C:\\Users\\Admin\\Downloads\\training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
    df.dropna(axis=1,inplace=True) df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
    df.head()
    temp = pd.read_csv("C:\\Users\\Admin\\Downloads\\Processed_data.csv")
    temp.drop("Unnamed: 0",inplace=True,axis=1)
    df['domain1_score']=temp['final_score']
    df.head()
    df['essay'][0]
    temp.head(1)
    #Make Dataset
    y = df['domain1_score']
    df.drop('domain1_score',inplace=True,axis=1)
    X=df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train.shape
    train_e = X_train['essay'].tolist()
    test_e = X_test['essay'].tolist()
    train_sents=[]
    test_sents=[]
    stop_words = set(stopwords.words('english'))
    def sent2word(x):
        x=re.sub("[^A-Za-z]"," ",x)
        x.lower()
        filtered_sentence = []
        words=x.split()
        for w in words:
            if w not in stop_words:
                filtered_sentence.append(w)
        return filtered_sentence
    def essay2word(essay):
        essay = essay.strip()
        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        raw = tokenizer.tokenize(essay)
        final_words=[]
        for i in raw:
            if(len(i)>0):
                final_words.append(sent2word(i))
        return final_words
    for i in train_e:
        train_sents+=essay2word(i)
    for i in test_e:
        test_sents+=essay2word(i)
    len(train_sents)
    train_sents[0]
    def get_model():
        model = Sequential()
        model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
        model.add(LSTM(64, recurrent_dropout=0.4))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='relu'))
        model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
        model.summary()
        return model
    #Training Word2Vec model
    num_features = 300
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3
    model = Word2Vec(train_sents,
                    workers=num_workers,
                    vector_size=num_features,  # Use 'vector_size' instead of 'size'
                    min_count=min_word_count,
                    window=context,
                    sample=downsampling)
    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
    def makeVec(words, model, num_features):
        vec = np.zeros((num_features,), dtype="float32")
        noOfWords = 0.0
        for i in words:
            if i in model.wv:
                noOfWords += 1
                vec = np.add(vec, model.wv.get_vector(i))
        if noOfWords > 0:
            vec = np.divide(vec, noOfWords)
        return vec
    def getVecs(essays, model, num_features):
        essay_vecs = np.zeros((len(essays), num_features), dtype="float32")
        for c, essay in enumerate(essays):
            essay_vecs[c] = makeVec(essay, model, num_features)
        return essay_vecs
    clean_train = []
    for i in train_e:
        clean_train.append(sent2word(i))
    training_vectors = getVecs(clean_train, model, num_features)
    clean_test = []
    for i in test_e:
        clean_test.append(sent2word(i))
    testing_vectors = getVecs(clean_test, model, num_features)
    training_vectors.shape
    training_vectors = np.array(training_vectors)
    testing_vectors = np.array(testing_vectors)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
    testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
    lstm_model = get_model()
    training_vectors.shape
    lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)
    loss,mea = lstm_model.evaluate(testing_vectors,y_test)
    lstm_model.save('final_lstm.h5')
    y_pred = lstm_model.predict(testing_vectors)
    y_pred = np.around(y_pred)
    y_pred
    return render(request,"users/training.html",{'loss':loss,'mean_absolute_error':mea}) #,{"Accuracy":eval_accuracy * 100,"Loss":eval_loss}
def prediction(request):
    if request.method=='POST':
        final_text = request.POST.get('final_text')
        from flask import Flask,request,render_template,url_for,jsonify
        import site
        import numpy as np
        import pandas as pd
        import nltk
        import re
        from nltk.corpus import stopwords
        from nltk.tokenize import sent_tokenize,word_tokenize
        from gensim.models import Word2Vec
        from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
        from keras.models import Sequential, load_model, model_from_config
        import keras.backend as K
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import cohen_kappa_score
        from gensim.models.keyedvectors import KeyedVectors
        from keras import backend as K
        def sent2word(x):
            stop_words = set(stopwords.words('english'))
            x=re.sub("[^A-Za-z]"," ",x)
            x.lower()
            filtered_sentence = []
            words=x.split()
            for w in words:
                if w not in stop_words:
                    filtered_sentence.append(w)
            return filtered_sentence
        def essay2word(essay):
            essay = essay.strip()
            tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
            raw = tokenizer.tokenize(essay)
            final_words=[]
            for i in raw:
                if(len(i)>0):
                    final_words.append(sent2word(i))
            return final_words
        def makeVec(words, model, num_features):
            vec = np.zeros((num_features,),dtype="float32")
            noOfWords = 0.
            index2word_set = set(model.index_to_key)
            for i in words:
                if i in index2word_set:
                    noOfWords += 1
                    vec = np.add(vec,model[i])
            vec = np.divide(vec,noOfWords)
            return vec
        def getVecs(essays, model, num_features):
            c=0
            essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
            for i in essays:
                essay_vecs[c] = makeVec(i, model, num_features)
                c+=1
            return essay_vecs
        def get_model():
            model = Sequential()
            model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
            model.add(LSTM(64, recurrent_dropout=0.4))
            model.add(Dropout(0.5))
            model.add(Dense(1, activation='relu'))
            model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
            model.summary()
            return model
        def convertToVec(text):
            content=text
            if len(content) > 20:
                num_features = 300
                model = KeyedVectors.load_word2vec_format("word2vecmodel.bin", binary=True)
                clean_test_essays = []
                clean_test_essays.append(sent2word(content))
                testDataVecs = getVecs(clean_test_essays, model, num_features )
                testDataVecs = np.array(testDataVecs)
                testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
                lstm_model = load_model("final_lstm.h5")
                preds = lstm_model.predict(testDataVecs)
                return str(round(preds[0][0]))
        def convertToVec(text):
            if text is not None and len(text.strip()) > 20:
                content = text
                num_features = 300
                model = KeyedVectors.load_word2vec_format("/content/Automatic-Essay-Scoring/word2vecmodel.bin", binary=True)
                clean_test_essays = []
                clean_test_essays.append(sent2word(content))
                testDataVecs = getVecs(clean_test_essays, model, num_features)
                testDataVecs = np.array(testDataVecs)
                testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
                lstm_model = load_model("/content/Automatic-Essay-Scoring/final_lstm.h5")
                preds = lstm_model.predict(testDataVecs)
                return str(round(preds[0][0]))
            else:
                # Handle the case where text is None or too short
                return "Unable to calculate a score."
        score = convertToVec(final_text)
        print(score)
        return render(request, 'users/predictForm.html', {'score':score})
    elsereturn render(request, 'users/predictForm.html', {})
base.html:
{% load static %}
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta content="width=device-width, initial-scale=1.0" name="viewport">
  <title>Arsha Bootstrap Template - Index</title>
  <meta content="" name="description">
  <meta content="" name="keywords">
  <!-- Favicons -->
  <link href="{% static 'assets/img/favicon.png' %}" rel="icon">
  <link href="{% static 'assets/img/apple-touch-icon.png' %}" rel="apple-touch-icon">
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,700,700i|Jost:300,300i,400,400i,500,500i,600,600i,700,700i|Poppins:300,300i,400,400i,500,500i,600,600i,700,700i" rel="stylesheet">
  <!-- Vendor CSS Files -->
  <link href="{% static 'assets/vendor/aos/aos.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/bootstrap/css/bootstrap.min.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/bootstrap-icons/bootstrap-icons.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/boxicons/css/boxicons.min.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/glightbox/css/glightbox.min.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/remixicon/remixicon.css' %}" rel="stylesheet">
  <link href="{% static 'assets/vendor/swiper/swiper-bundle.min.css' %}" rel="stylesheet">
  <!-- Template Main CSS File -->
  <link href="{% static 'assets/css/style.css' %}" rel="stylesheet">
  <!-- =======================================================
  * Template Name: Arsha - v4.11.0
  * Template URL: https://bootstrapmade.com/arsha-free-bootstrap-html-template-corporate/
  * Author: BootstrapMade.com
  * License: https://bootstrapmade.com/license/
  ======================================================== -->
</head>
<body>
    <style>
        span{
            color:white;
        }
        header{
            background-color:GREEN;
        }
        body{
            background-image: url('{% static 'assets/img/essay.jpg' %}');
        }
    </style>
  <!-- ======= Header ======= -->
  <header id="header" class="fixed-top ">
    <div class="container d-flex align-items-center">
      <h1 class="logo me-auto"><a href="index.html"><span>Automatic English Essay Scoring</span></a></h1>
      <!-- Uncomment below if you prefer to use an image logo -->
      <!-- <a href="index.html" class="logo me-auto"><img src="assets/img/logo.png" alt="" class="img-fluid"></a>-->
      <nav id="navbar" class="navbar">
        <ul>
          <li><a class="nav-link scrollto active" href="{% url 'index' %}">Home</a></li>
          <li><a class="nav-link scrollto" href="{% url 'AdminLogin' %}">Admin</a></li>
          <li><a class="nav-link scrollto" href="{% url 'UserLogin' %}">User</a></li>
          <li><a class="nav-link   scrollto" href="{% url 'UserRegister' %}">Register</a></li>
        </ul>
        <i class="bi bi-list mobile-nav-toggle"></i>
      </nav><!-- .navbar -->
    </div>
  </header><!-- End Header -->
{% block contents %}
{% endblock %}
  <!-- ======= Footer ======= -->
  <footer id="footer">
  </footer><!-- End Footer -->
  <div id="preloader"></div>
  <a href="#" class="back-to-top d-flex align-items-center justify-content-center"><i class="bi bi-arrow-up-short"></i></a>
  <!-- Vendor JS Files -->
  <script src="{% static 'assets/vendor/aos/aos.js' %}"></script>
  <script src="{% static 'assets/vendor/bootstrap/js/bootstrap.bundle.min.js' %}"></script>
  <script src="{% static 'assets/vendor/glightbox/js/glightbox.min.js' %}"></script>
  <script src="{% static 'assets/vendor/isotope-layout/isotope.pkgd.min.js' %}"></script>
  <script src="{% static 'assets/vendor/swiper/swiper-bundle.min.js' %}"></script>
  <script src="{% static 'assets/vendor/waypoints/noframework.waypoints.js' %}"></script>
  <script src="{% static 'assets/vendor/php-email-form/validate.js' %}"></script>
  <!-- Template Main JS File -->
  <script src="{% static 'assets/js/main.js' %}"></script>
</body>
</html>
Admin side views:
from django.shortcuts import render
from django.contrib import messages
from users.forms import UserRegistrationForm
from users.models import UserRegistrationModel
# Create your views here.
def AdminLoginCheck(request):
    if request.method == 'POST':
        usrid = request.POST.get('loginid')
        pswd = request.POST.get('pswd')
        print("User ID is = ", usrid)
        if usrid == 'admin' and pswd == 'admin':
            return render(request, 'admins/AdminHome.html')
        else:
            messages.success(request, 'Please Check Your Login Details')
    return render(request, 'AdminLogin.html', {})
def AdminHome(request):
    return render(request, 'admins/AdminHome.html',{})
def RegisterUsersView(request):
    data = UserRegistrationModel.objects.all()
    return render(request,'admins/viewregisterusers.html',{'data':data})
def ActivaUsers(request):
    if request.method == 'GET':
        id = request.GET.get('uid')
        status = 'activated'
        print("PID = ", id, status)
        UserRegistrationModel.objects.filter(id=id).update(status=status)
        data = UserRegistrationModel.objects.all()
        return render(request,'admins/viewregisterusers.html',{'data':data})

